## Raw parquet file reading

In [ ]:
val vcf = "/tmp/6-sample.vcf"
val localParquet = vcf+"6"+".parquet"

In [ ]:
val rawDF = sparkSession.read.parquet(localParquet)

In [ ]:
// populations to select
val pops = Set("GBR", "ASW", "CHB")

In [ ]:
import sys.process._

val panelFile = "/tmp/ALL.panel"

s"wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/integrated_call_samples_v3.20130502.ALL.panel -O ${panelFile}"!!

In [ ]:
import scala.io.Source
def extract(filter: (String, String) => Boolean= (s, t) => true) = Source.fromFile(panelFile).getLines().map( line => {
  val toks = line.split("\t").toList
  toks(0) -> toks(1)
}).toMap.filter( tup => filter(tup._1, tup._2) )
  
// panel extract from file, filtering by the 2 populations
val panel: Map[String,String] = 
  extract((sampleID: String, pop: String) => pops.contains(pop)) 
  
val panelSamples = panel.keys.toList
// broadcast the panel 
val bPanel = sparkContext.broadcast(panel)

In [ ]:
panelSamples

In [ ]:
val finalDF = rawDF.filter($"sampleId" isin(panelSamples:_*))

In [ ]:
finalDF

In [ ]:
val sampleCount = finalDF.select($"sampleId").distinct.count
s"#Samples: $sampleCount"

In [ ]:
val selectedVariants = finalDF.groupBy($"variantId").count.filter($"count" === sampleCount).select("variantId")

In [ ]:
val completeDF = finalDF.join(selectedVariants, finalDF("variantId") === selectedVariants("variantId"))
                        .select($"sampleId",finalDF("variantId"),$"genotype")

In [ ]:
:sh rm -rf /tmp/genome.flat.parquet

In [ ]:
completeDF.cache()
completeDF.write.parquet("/tmp/genome.flat.parquet")

### Pivot with RDD API...

In [ ]:
val groupedRDD = completeDF.rdd.map{ r =>
             (r.getAs[String]("sampleId"), (r.getAs[Double]("genotype"), r.getAs[String]("variantId")))
              }.groupByKey

In [ ]:
import org.apache.spark.mllib.linalg.{Vector=>MLVector, Vectors}

def makeSortedVector(gts: Iterable[(Double, String)]): MLVector = Vectors.dense( gts.toArray.sortBy(_._2).map(_._1) )

val dataPerSampleId:RDD[(String, MLVector)] =
    groupedRDD.mapValues { it =>
        makeSortedVector(it)
    }

val dataFrame:RDD[MLVector] = dataPerSampleId.values

In [ ]:
import org.apache.spark.mllib.clustering.{KMeans,KMeansModel}
val model: KMeansModel = KMeans.train(dataFrame, 3, 10)

In [ ]:
val predictions: RDD[(String, (Int, String))] = dataPerSampleId.map(elt => {
    (elt._1, ( model.predict(elt._2), bPanel.value.getOrElse(elt._1, ""))) 
})

In [ ]:
val confMat = predictions.collect.toMap.values
    .groupBy(_._2).mapValues(_.map(_._1))
    .mapValues(xs => (1 to 3).map( i => xs.count(_ == i-1)).toList)

In [ ]:
<table>
<tr><td></td><td>#0</td><td>#1</td><td>#2</td></tr>
{ for (popu <- confMat) yield
  <tr><td>{popu._1}</td> { for (cnt <- popu._2) yield 
    <td>{cnt}</td>
  }
  </tr>
}
</table>

In [ ]:
import org.apache.spark.ml.feature.PCA
val seqAsVector = udf((xs: Seq[Double]) => org.apache.spark.ml.linalg.Vectors.dense(xs.toArray))
val df = dataFrame.map(_.toArray.take(1000)).toDF("array").withColumn("features", seqAsVector($"array"))

In [ ]:
val pca = new PCA()
  .setInputCol("features")
  .setOutputCol("pcaFeatures")
  .setK(3)
  .fit(df)

In [ ]:
val result = pca.transform(df).select("pcaFeatures")

In [ ]:
result.take(5)

In [ ]:
import org.apache.spark.ml.clustering.KMeans

In [ ]:
val kmeans = new org.apache.spark.ml.clustering.KMeans().setK(3).setFeaturesCol("pcaFeatures").setSeed(1L)
val model = kmeans.fit(result)

### TODO estimate a confusion matrix?